In [1]:
import os
os.chdir("/")

from google.colab import drive
try:
    drive.mount('content/gdrive', force_remount=True)
except:
    print('你的 drive 掛載不成功')
    
os.chdir("/content/gdrive/My Drive/")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at content/gdrive


In [2]:
files = []
for r, d, f in os.walk("/content/gdrive/My Drive/GPT2-Chinese-master/"):
    for file in f:
        
        files.append(os.path.join(r, file))

for f in files:
    print(f)

os.chdir("/content/gdrive/My Drive/GPT2-Chinese-master/")

/content/gdrive/My Drive/GPT2-Chinese-master/train_single.py
/content/gdrive/My Drive/GPT2-Chinese-master/LICENSE
/content/gdrive/My Drive/GPT2-Chinese-master/requirements.txt
/content/gdrive/My Drive/GPT2-Chinese-master/generate.py
/content/gdrive/My Drive/GPT2-Chinese-master/train.json
/content/gdrive/My Drive/GPT2-Chinese-master/README.md
/content/gdrive/My Drive/GPT2-Chinese-master/.gitignore
/content/gdrive/My Drive/GPT2-Chinese-master/generate_texts.py
/content/gdrive/My Drive/GPT2-Chinese-master/eval.py
/content/gdrive/My Drive/GPT2-Chinese-master/train.py
/content/gdrive/My Drive/GPT2-Chinese-master/實驗.gsheet
/content/gdrive/My Drive/GPT2-Chinese-master/run_GPT.ipynb
/content/gdrive/My Drive/GPT2-Chinese-master/sample/doupo.jpeg
/content/gdrive/My Drive/GPT2-Chinese-master/sample/散文3.png
/content/gdrive/My Drive/GPT2-Chinese-master/sample/金庸_神鵰俠侶.jpg
/content/gdrive/My Drive/GPT2-Chinese-master/sample/散文1.png
/content/gdrive/My Drive/GPT2-Chinese-master/sample/律诗绝句.png
/content

0.

In [3]:
!pip install transformers==2.1.1
#!pip install sentencepiece
#!pip install tensorflow_addons
!pip install hanziconv

     |████████████████████████████████| 317kB 8.9MB/s 
     |████████████████████████████████| 1.0MB 60.5MB/s 
     |████████████████████████████████| 870kB 63.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884629 sha256=f3a83dc2c9f9bd47e380591e61a0c69acef15e7aa656d2a5d039953d1cf5ed08
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses
     |████████████████████████████████| 286kB 9.5MB/s 
  Created wheel for hanziconv: filename=hanziconv-0.3.2-py2.py3-none-any.whl size=23215 sha256=988faba4271dfb78a7fd14059f202778ef96018e39e8ee27d29dba1e0b70f421
  Stored in directory: /root/.cache/pip/wheels/03/d8/3c/c39898fa9c9ce6e34b0ab4c6604892462d440c743715c94054
Successfully built hanziconv


# 1. 處理輸入資料

In [0]:
import re
from hanziconv import HanziConv
import json
import random


In [0]:
def remove_punctuation(line):

    #rule = re.compile("[^a-zA-Z0-9\u4e00-\u9fa5]")


    line = line.replace("\\n","").replace("\n","")
    #
    rule = re.compile("[\[|\]|\"|\']")
    line = rule.sub('',line)

    #
    rule = re.compile("[　| ]")
    line = rule.sub(',',line)

    #
    line = HanziConv.toTraditional(line)
    return line

In [0]:

output = []
for r, d, f in os.walk("/content/gdrive/My Drive/GPT2-Chinese-master/data/wiki_zh/"):
    for file in f:
        if file.startswith('wiki'):
            print(r+'/'+file)
            with open(r+'/'+file,"r") as f:
                lines = f.readlines()
                for line in lines:
                    sentence = json.loads(line)
                    #print(sentence.keys())
                    templine = remove_punctuation(sentence['text'])
                    #print(templine)
                    output.append(templine)

random.shuffle(output)
with open("./data/train.json","w") as f:
    print("len",len(output))
    json.dump(output,f,ensure_ascii=False)

weight = 0.00
for i in range(20):
    weight+= 0.05
    with open("./data/train_{}.json".format(int(weight*100)),"w") as f:
        count = int(len(output)*weight)
        print("len",count)
        json.dump(output[:count],f,ensure_ascii=False)



len 625934
len 678095
len 730256
len 782418
len 834579
len 886740
len 938901
len 991062
len 1043224


In [0]:
with open("./data/train_source.txt","r") as f:
    lines = f.readlines()

output = []
for idx,line in enumerate(lines):
    #print(line)
    templine = remove_punctuation(line)
    output.append(templine)
print("len", len(output))
#output = str(output).replace("\'","\"")
with open("./data/train.json","w") as f:
    json.dump(output,f,ensure_ascii=False)

len 500


# 2. 訓練

In [0]:
!python train.py \
  --raw_data_path data/train_0_05.json \
  --epochs 5 \
  --output_dir model/ \
  --batch_size 16 \
  --min_length 100 \
  --warmup_steps 20000 \
  --writer_dir model/tensorboard/ \
  --stride 256 \
  > output/result_D005_S16.txt

100% 100/100 [00:20<00:00,  1.90it/s]


# 3.產生


In [5]:
!python ./generate.py   \
  --length=200   \
  --nsamples=8    \
  --prefix=立法院三讀通過總統要吃一百個漢堡   \
  --fast_pattern   \
  --save_samples   \
  --save_samples_path=/output


args:
Namespace(batch_size=1, device='0,1,2,3', fast_pattern=True, length=200, model_config='config/model_config_small.json', model_path='model/final_model', no_wordpiece=False, nsamples=8, prefix='立法院三讀通過總統要吃一百個漢堡', repetition_penalty=1.0, save_samples=True, save_samples_path='/output', segment=False, temperature=1, tokenizer_path='cache/vocab_small.txt', topk=8, topp=0)
100% 200/200 [00:02<00:00, 81.38it/s]
======================================== SAMPLE 1 ========================================

立法院三讀通過總統要吃一百個漢堡的法令，並要求總統要求國防部長以及各方的代議案進行決定，並要求總統要求總統必須提交總統的建議。2013年6月12日，總統選舉産生瞭148個議案，總統的任命範圍為334.8334384246432.,總統選舉産生瞭302個議案。總統選舉産生的黨員中，有72個議員，185個議員，58個黨員，123個非政府組織，13個國防組織和256個非政府組織。總統選舉産生的黨員中，有34個政黨，128個非政府組織，
100% 200/200 [00:02<00:00, 80.86it/s]
======================================== SAMPLE 2 ========================================

立法院三讀通過總統要吃一百個漢堡的方案，以確保總統的選舉過多，也因此被認為是一個“不可能的”，因為他不希望這樣做得到瞭總統的支持，並且不斷提齣這一提案。在1952年7月25日，國會通過瞭《1952年法案》，並通過瞭《1952年法案》。在第五屆國會中，國會通過瞭《1952年憲法》，其中規定“不可能